In [3]:
# 📦 Imports
from fredapi import Fred
import yfinance as yf
import pandas as pd
import sys
import os

# Go up one level to find linchpin_functions/
parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from linchpin_functions import fill_or_impute_real_gdp

# 🔧 Setup
API_KEY = "YOUR API KEY HERE"
fred = Fred(api_key=API_KEY)
start_date = "2000-01-01"
end_date = "2025-01-01"
freq = "ME"  # Monthly end for resampling

# ✅ Feature mapping: FRED/Yahoo Finance series → custom names
features = {
    "NAPMNIM": "linchpin__ism_nonmanufacturing_index",
    "MORTGAGE30US": "linchpin__mortgage_rate",
    "CPILFESL": "linchpin__core_cpi",
    "IR14270": "linchpin__import_prices",
    "IQEXPT": "linchpin__export_prices",  # May fail — fallback used
    "RSAFS": "linchpin__retail_sales",
    "INDPRO": "linchpin__industrial_production",
    "PAYEMS": "linchpin__nonfarm_payrolls",
    "GDPC1": "linchpin__real_gdp_growth",
    "GS10": "linchpin__10y_treasury_yield"
}

# 🗓️ Base monthly date index
date_index = pd.date_range(start=start_date, end=end_date, freq="ME")
df = pd.DataFrame(index=date_index)

# 📥 Pull FRED data
for code, alias in features.items():
    try:
        series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
        series = series.resample("ME").ffill()  # Align to month-end
        df[alias] = series
        print(f"✅ Loaded {alias}")
    except Exception as e:
        print(f"❌ Failed to load {alias}: {e}")

# 📈 Add S&P 500 returns (via yfinance)
try:
    sp500 = yf.download("^GSPC", start=start_date, end=end_date, interval="1mo", auto_adjust=True)
    sp500.index = pd.to_datetime(sp500.index).to_period("M").to_timestamp("M")  # Align to month-end
    sp500_returns = sp500["Close"].pct_change()
    # df["linchpin__s&p500_returns"] = sp500_returns.reindex(df.index)
    df["linchpin__s&p500_returns"] = sp500_returns.reindex(df.index).bfill()
    print("✅ Loaded linchpin__s&p500_returns via yfinance")
except Exception as e:
    print(f"❌ Failed to fetch S&P 500 returns: {e}")

# 🧠 Engineer fallback for ISM non-manufacturing index
try:
    df["linchpin__ism_nonmanufacturing_index"] = (
        df["linchpin__retail_sales"].pct_change(fill_method=None) * 0.4 +
        df["linchpin__nonfarm_payrolls"].pct_change(fill_method=None) * 0.3 +
        df["linchpin__core_cpi"].pct_change(fill_method=None) * 0.3
    )
    # Impute first missing value if exists
    if df["linchpin__ism_nonmanufacturing_index"].isna().sum() > 0:
        df["linchpin__ism_nonmanufacturing_index"] = df["linchpin__ism_nonmanufacturing_index"].bfill()
        print("🩹 Imputed missing value in linchpin__ism_nonmanufacturing_index with bfill")
    print("✅ Engineered linchpin__ism_nonmanufacturing_index from proxy components")
except Exception as e:
    print(f"❌ Failed to engineer linchpin__ism_nonmanufacturing_index: {e}")

# 🧠 Engineer fallback for export prices
try:
    df["linchpin__export_prices"] = (
        df["linchpin__import_prices"].pct_change(fill_method=None) * 0.5 +
        df["linchpin__core_cpi"].pct_change(fill_method=None) * 0.3 +
        df["linchpin__10y_treasury_yield"].pct_change(fill_method=None) * 0.2
    )
    if df["linchpin__export_prices"].isna().sum() > 0:
        df["linchpin__export_prices"] = df["linchpin__export_prices"].bfill()
        print("🩹 Imputed missing value in linchpin__export_prices with bfill")
    print("✅ Engineered linchpin__export_prices from proxy components")
except Exception as e:
    print(f"❌ Failed to engineer linchpin__export_prices: {e}")

# 🧼 Final formatting
# 🧼 Monthly alignment using forward-fill (preferred for economics)
df.index.name = "date"
df = df.resample("ME").ffill()

df = fill_or_impute_real_gdp(
    df,
    predictor_cols=[
        "linchpin__core_cpi",
        "linchpin__retail_sales",
        "linchpin__nonfarm_payrolls"
    ]
)

# 💾 Save to CSV
os.makedirs("linchpin_batches", exist_ok=True)
df.to_csv("../linchpin_batches/Batch_Linchpin_02.csv")
print("✅ Saved Batch_Linchpin_02.csv")

❌ Failed to load linchpin__ism_nonmanufacturing_index: Bad Request.  The series does not exist.
✅ Loaded linchpin__mortgage_rate
✅ Loaded linchpin__core_cpi
✅ Loaded linchpin__import_prices
❌ Failed to load linchpin__export_prices: Bad Request.  The series does not exist.
✅ Loaded linchpin__retail_sales
✅ Loaded linchpin__industrial_production
✅ Loaded linchpin__nonfarm_payrolls
✅ Loaded linchpin__real_gdp_growth


[*********************100%***********************]  1 of 1 completed

✅ Loaded linchpin__10y_treasury_yield
✅ Loaded linchpin__s&p500_returns via yfinance
🩹 Imputed missing value in linchpin__ism_nonmanufacturing_index with bfill
✅ Engineered linchpin__ism_nonmanufacturing_index from proxy components
🩹 Imputed missing value in linchpin__export_prices with bfill
✅ Engineered linchpin__export_prices from proxy components
✅ Saved Batch_Linchpin_02.csv
